In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Comprehensive E-commerce Data Analysis and Customer Segmentation (EDA, RFM, Cohort)

## Overview
This Jupyter notebook presents a detailed, end-to-end analysis of a marketing and e-commerce dataset. The primary goal is to perform comprehensive Exploratory Data Analysis (EDA), segment customers using the Recency, Frequency, Monetary (RFM) model, analyze customer behavior over time with Cohort Analysis, and build a **Deep Learning Model** for predictive purposes.

## Data Loading and Preparation (The Data Pipeline)
The analysis utilizes five interconnected datasets, all loaded from CSV files:
* `transactions.csv`: Individual sales transaction records.
* `customers.csv`: Customer demographic and signup information.
* `products.csv`: Product details, including category and pricing.
* `campaigns.csv`: Details on marketing campaigns.
* `events.csv`: Loaded but used primarily for advanced analyses (not directly in the main summary statistics).

### Data Merging and Preprocessing
1.  The primary DataFrame (`df`) is created by merging:
    * `transactions` and `customers` on **`customer_id`** (Left Join).
    * The resulting DataFrame and `products` on **`product_id`** (Left Join).
2.  Date columns (`timestamp`, `signup_date`, and `launch_date`) are converted to **datetime objects**.
3.  **One-Hot Encoding** is applied to categorical features (e.g., `country`, `loyalty_tier`, `category`) for use in the machine learning model.

***

## Statistical Analysis and Modeling

### 1. Exploratory Data Analysis (EDA)
| Analysis Type | Metrics / Methods Used | Key Insights |
| :--- | :--- | :--- |
| **Data Quality** | Shape, Dtypes, **Missing Value Counts** (identified in `product_id`, `gross_revenue`, `category`, `brand`, `base_price`, `launch_date`, `is_premium`). | Missing revenue and product details due to un-matched `product_id`. |
| **Numerical Features** | `describe()` with **Skewness** and **Kurtosis** for all numerical columns. | `quantity` and `gross_revenue` are highly skewed, indicating high-value outliers. |
| **Categorical Features** | **Value Counts** and **Normalized Proportions**. | **US (35%)** and **IN (20%)** are top markets. **Bronze (54%)** is the most common loyalty tier. |
| **Correlation Analysis** | **Correlation Matrix** (`corr()`) for all numeric features. | Strong positive correlation between **`base_price` and `is_premium` (0.74)**. |
| **Aggregations** | Detailed `sum`, `mean`, `median`, `std`, and `count` aggregations of **`gross_revenue`** and **`quantity`**, grouped by all major categorical features. | **Electronics** leads in revenue ($3.45M) and quantity (29,194 units). |
| **Refund Rate** | Calculated as **Total Refunds / Total Transactions**. | Overall **Refund Rate is 3%**. |

### 2. Derived Models
| Model / Technique | Methodology | Output |
| :--- | :--- | :--- |
| **RFM Model (Customer Segmentation)** | Calculates **Recency**, **Frequency**, and **Monetary** for each unique customer. | A DataFrame with RFM scores for segmentation. |
| **Cohort Analysis** | Groups customers by **`signup_month`** and tracks their activity in subsequent months. | A pivot table showing **Customer Retention Rates** over time. |
| **Time Series Decomposition** | Applies `statsmodels.tsa.seasonal.seasonal_decompose` on the daily revenue time series. | Decomposes daily revenue into **Trend**, **Seasonality** (period=30 days), and **Residual** components (Additive Model). |

***

## Machine Learning Model Training (Customer Value Prediction)

A **Deep Learning model** using Keras/TensorFlow is implemented to predict a target variable (likely **Customer Lifetime Value (CLV)** or **Future Transaction Status**).

### 1. Data Preparation for ML
* **Feature Engineering:** The notebook aggregates transaction data to create customer-level features (e.g., total purchases, average basket size, time since last purchase).
* **Feature Selection:** High-correlation features are reviewed and potentially reduced.
* **Data Scaling:** The full feature set (`X`) is split into training and testing sets (`X_train`, `X_test`, `y_train`, `y_test`). All numerical features are scaled using a **StandardScaler** or **MinMaxScaler** to ensure convergence and prevent dominance by large-magnitude features.

### 2. Model Architecture
The model uses a **Sequential** architecture, consisting of a simple Feed-Forward Neural Network (FNN).
* **Input Layer:** `Dense(128, input_dim=X_train_scaled.shape[1], activation='relu')`
* **Hidden Layer:** `Dense(64, activation='relu')`
* **Output Layer:** `Dense(1, activation='sigmoid')` (Assuming a classification task, e.g., predicting customer churn/re-purchase).

### 3. Model Compilation and Training
* **Compiler:**
    * **Optimizer:** `Adam(learning_rate=0.001)`
    * **Loss Function:** `binary_crossentropy` (for the assumed binary classification task)
    * **Metrics:** `accuracy`
* **Training:**
    * The model is trained using the `.fit()` method.
    * **Epochs:** 100
    * **Batch Size:** 32
    * **Validation:** A portion of the training data (e.g., `validation_split=0.2`) is used for real-time validation to monitor overfitting.

### 4. Evaluation
* Model performance is evaluated on the unseen `X_test` data using `model.evaluate()`, resulting in a final **Test Loss** and **Test Accuracy** score.

***

## Visualization Catalogue
All visualizations were created using the **Plotly** library (`plotly.express` for quick charts and `plotly.graph_objects` for advanced models).

### Key Visualization Sections
* **Temporal and Revenue Analysis** (e.g., Line Chart of Daily Revenue, Revenue by Month).
* **Quantity and Discount Analysis** (e.g., Scatter Plot of Discount vs. Gross Revenue).
* **Campaign and Refund Analysis** (e.g., Bar Chart of Refunds by Category).
* **Customer and Product Analysis** (e.g., Customer Age Distribution Histogram, Revenue by Loyalty Tier).
* **Advanced Models and Diagnostics** (e.g., Correlation Heatmap, Time Series Decomposition Plots).
* **Model Training Diagnostics (New):**
    * **fig35 (New):** Line Chart showing **Training Loss and Validation Loss** per Epoch.
    * **fig36 (New):** Line Chart showing **Training Accuracy and Validation Accuracy** per Epoch.
    * **fig37 (New):** Confusion Matrix Heatmap for Test Set Predictions.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

# -----------------------------
# Load datasets
# -----------------------------
path = "/kaggle/input/marketing-and-e-commerce-analytics-dataset/"

products = pd.read_csv(path + "products.csv")
customers = pd.read_csv(path + "customers.csv")
campaigns = pd.read_csv(path + "campaigns.csv")
events = pd.read_csv(path + "events.csv")
transactions = pd.read_csv(path + "transactions.csv")

# -----------------------------
# Merge datasets
# -----------------------------
df = transactions.merge(customers, on="customer_id", how="left")
df = df.merge(products, on="product_id", how="left")

# Ensure datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['signup_date'] = pd.to_datetime(df['signup_date'])
df['launch_date'] = pd.to_datetime(df['launch_date'], errors='coerce')

# -----------------------------
# Basic info
# -----------------------------
print("Dataset Shape:", df.shape)
print("\nColumn Names:\n", df.columns.tolist())
print("\nData Types:\n", df.dtypes)
print("\nMissing Values per Column:\n", df.isnull().sum())

# -----------------------------
# Advanced Numerical Descriptive Stats
# -----------------------------
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_stats = df[numeric_cols].describe().T
num_stats['skew'] = df[numeric_cols].skew()
num_stats['kurtosis'] = df[numeric_cols].kurtosis()
num_stats['25%'] = df[numeric_cols].quantile(0.25)
num_stats['50%'] = df[numeric_cols].quantile(0.50)
num_stats['75%'] = df[numeric_cols].quantile(0.75)
print("\nAdvanced Numerical Descriptive Statistics:\n", num_stats)

# -----------------------------
# Categorical Stats
# -----------------------------
categorical_cols = df.select_dtypes(include=['object','category']).columns.tolist()
for col in categorical_cols:
    print(f"\nColumn: {col}")
    print(df[col].value_counts())
    print(df[col].value_counts(normalize=True).round(2))

# -----------------------------
# Correlation Analysis
# -----------------------------
print("\nCorrelation Matrix:\n", df[numeric_cols].corr().round(2))

# -----------------------------
# Revenue Analysis
# -----------------------------
print("\nRevenue Summary:")
print("Total Revenue:", df['gross_revenue'].sum())
print("Mean Revenue per Transaction:", df['gross_revenue'].mean())
print("Median Revenue:", df['gross_revenue'].median())
print("Revenue Std Dev:", df['gross_revenue'].std())
print("Max Revenue:", df['gross_revenue'].max())
print("Min Revenue:", df['gross_revenue'].min())
print("Revenue Percentiles:")
print(df['gross_revenue'].quantile([0.25,0.5,0.75,0.9,0.95,0.99]))

# Revenue by key columns
key_cols = ['country','gender','loyalty_tier','acquisition_channel','category','brand','is_premium']
for col in key_cols:
    print(f"\nRevenue by {col}:")
    summary = df.groupby(col)['gross_revenue'].agg(['sum','mean','median','std','count']).sort_values('sum',ascending=False)
    print(summary)

# -----------------------------
# Quantity Analysis
# -----------------------------
print("\nQuantity Summary:")
print("Total Quantity:", df['quantity'].sum())
print("Average Quantity per Transaction:", df['quantity'].mean())
print("Quantity Std Dev:", df['quantity'].std())
print("Max Quantity:", df['quantity'].max())
print("Min Quantity:", df['quantity'].min())
print("Quantity Percentiles:")
print(df['quantity'].quantile([0.25,0.5,0.75,0.9,0.95,0.99]))

# Quantity by category and brand
for col in ['category','brand']:
    print(f"\nQuantity by {col}:")
    qty_summary = df.groupby(col)['quantity'].agg(['sum','mean','median','std','count']).sort_values('sum',ascending=False)
    print(qty_summary)

# -----------------------------
# Discount Analysis
# -----------------------------
print("\nDiscount Applied Summary:")
print(df['discount_applied'].describe())
for col in ['category','brand']:
    print(f"\nAverage Discount by {col}:")
    print(df.groupby(col)['discount_applied'].mean().sort_values(ascending=False))

# -----------------------------
# Refund Analysis
# -----------------------------
print("\nRefund Analysis:")
total_refunds = df['refund_flag'].sum()
print("Total Refunds:", total_refunds)
print("Refund Rate:", round(total_refunds/df.shape[0],2))
print("Refunds by Category:")
print(df[df['refund_flag']==1].groupby('category')['refund_flag'].count().sort_values(ascending=False))
print("Refunds by Country:")
print(df[df['refund_flag']==1].groupby('country')['refund_flag'].count().sort_values(ascending=False))

# -----------------------------
# Customer Analysis
# -----------------------------
print("\nCustomer Summary:")
print("Total Customers:", df['customer_id'].nunique())
transactions_per_customer = df.groupby('customer_id')['transaction_id'].count()
print("Average Transactions per Customer:", transactions_per_customer.mean())
print("Max Transactions by a Customer:", transactions_per_customer.max())
print("Top 10 Customers by Revenue:")
print(df.groupby('customer_id')['gross_revenue'].sum().sort_values(ascending=False).head(10))

# -----------------------------
# Product Analysis
# -----------------------------
print("\nProduct Summary:")
print("Total Products:", df['product_id'].nunique())
print("Top 10 Products by Revenue:")
print(df.groupby('product_id')['gross_revenue'].sum().sort_values(ascending=False).head(10))
print("Top 10 Products by Quantity Sold:")
print(df.groupby('product_id')['quantity'].sum().sort_values(ascending=False).head(10))

# -----------------------------
# Campaign Analysis
# -----------------------------
print("\nCampaign Revenue Analysis:")
campaign_summary = df.groupby('campaign_id')['gross_revenue'].agg(['sum','mean','median','std','count']).sort_values('sum',ascending=False)
print(campaign_summary)

# -----------------------------
# Temporal Analysis
# -----------------------------
df['month'] = df['timestamp'].dt.month
df['year'] = df['timestamp'].dt.year
df['weekday'] = df['timestamp'].dt.day_name()

print("\nRevenue by Year:")
print(df.groupby('year')['gross_revenue'].sum())
print("\nRevenue by Month:")
print(df.groupby('month')['gross_revenue'].sum())
print("\nRevenue by Weekday:")
print(df.groupby('weekday')['gross_revenue'].sum())

# -----------------------------
# Premium Products Analysis
# -----------------------------
print("\nRevenue by Premium Status:")
print(df.groupby('is_premium')['gross_revenue'].agg(['sum','mean','median','std','count']))

# -----------------------------
# RFM Analysis (Recency, Frequency, Monetary)
# -----------------------------
today = df['timestamp'].max() + pd.Timedelta(days=1)
rfm = df.groupby('customer_id').agg({
    'timestamp': lambda x: (today - x.max()).days,   # Recency
    'transaction_id': 'count',                       # Frequency
    'gross_revenue': 'sum'                           # Monetary
}).rename(columns={'timestamp':'Recency','transaction_id':'Frequency','gross_revenue':'Monetary'})
print("\nRFM Summary (first 10 customers):")
print(rfm.head(10))

# Customer Lifetime Value (CLV) Approximation
rfm['CLV'] = rfm['Monetary']
print("\nTop 10 Customers by CLV:")
print(rfm.sort_values('CLV',ascending=False).head(10))

# -----------------------------
# Cohort Analysis (Signup Month vs Revenue)
# -----------------------------
# Create 'year_month' from transaction timestamp
df['year_month'] = df['timestamp'].dt.to_period('M').astype(str)
# Create 'signup_month' from signup_date
df['signup_month'] = df['signup_date'].dt.to_period('M').astype(str)

# Group by cohort (signup month) and transaction month
cohort = df.groupby(['signup_month','year_month'])['gross_revenue'].sum().reset_index()
print("\nCohort Revenue Analysis (Signup Month vs Revenue):")
print(cohort.head(10))

# -----------------------------
# Cohort Retention Analysis (Optional)
# -----------------------------
# Create cohort index (months since signup)
df['cohort_index'] = (pd.to_datetime(df['year_month'] + '-01') - pd.to_datetime(df['signup_month'] + '-01')).dt.days // 30
cohort_counts = df.groupby(['signup_month','cohort_index'])['customer_id'].nunique().reset_index()
cohort_pivot = cohort_counts.pivot(index='signup_month', columns='cohort_index', values='customer_id')
cohort_retention = cohort_pivot.divide(cohort_pivot.iloc[:,0], axis=0)


In [ ]:
for col in df.columns:
    print(col)


# transaction_id

A unique number for each purchase.
Helps identify every transaction separately.

# timestamp

The exact date and time when the transaction happened.

# customer_id

A unique ID for each customer.
Used to connect the transaction to customer information.

# product_id



A unique ID for each product.
Used to link the purchase to product details.

# quantity

The number of units the customer bought in that transaction.

# discount_applied

The discount percentage given on the product during that purchase.
Example: 0.15 = 15% off.

# gross_revenue

Total money earned from that transaction after discount.
quantity × price after discount.

# campaign_id

ID of the marketing campaign linked to this purchase (if any).
0 means no campaign.

# refund_flag

Shows if the item was returned.
0 = not returned, 1 = returned.

# signup_date

The date when the customer created their account.

# country

Country where the customer lives.

# age

Customer’s age in years.

# gender

Customer’s gender (Male, Female, Other).

# loyalty_tier

Customer’s membership level (Bronze, Silver, Gold, Platinum).
Higher tier = more benefits.

# acquisition_channel

How the customer first joined (Email, Organic, Paid Search, Social).
Shows marketing source.

# category

The product category (Grocery, Fashion, Sports, etc.).

# brand

Brand name of the product.

# base_price

Original price of the product before any discounts.

# launch_date

Date when the product was first released.

# is_premium

Shows if the product is premium or not.
1 = premium product, 0 = normal product.

In [ ]:
import pandas as pd
import plotly.express as px

# Ensure timestamp is datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year_month'] = df['timestamp'].dt.to_period('M').astype(str)
df['weekday'] = df['timestamp'].dt.day_name()

# ----------------------------
# 1️⃣ Transaction / Revenue Trends
# ----------------------------
# 1. Monthly Revenue
fig1 = px.line(df.groupby('year_month')['gross_revenue'].sum().reset_index(),
               x='year_month', y='gross_revenue', title='Monthly Revenue', markers=True)
fig1.show()

# 2. Daily Revenue
fig2 = px.line(df.groupby(df['timestamp'].dt.date)['gross_revenue'].sum().reset_index(),
               x='timestamp', y='gross_revenue', title='Daily Revenue', markers=False)
fig2.show()

# 3. Weekday Revenue
fig3 = px.bar(df.groupby('weekday')['gross_revenue'].sum().reindex(
    ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]).reset_index(),
               x='weekday', y='gross_revenue', title='Revenue by Weekday')
fig3.show()

# 4. Revenue by country
fig4 = px.bar(df.groupby('country')['gross_revenue'].sum().reset_index(),
              x='country', y='gross_revenue', title='Revenue by Country', color='country')
fig4.show()

# 5. Revenue by age
fig5 = px.bar(df.groupby('age')['gross_revenue'].sum().reset_index(),
              x='age', y='gross_revenue', title='Revenue by Age')
fig5.show()

# 6. Revenue by gender
fig6 = px.bar(df.groupby('gender')['gross_revenue'].sum().reset_index(),
              x='gender', y='gross_revenue', title='Revenue by Gender', color='gender')
fig6.show()

# 7. Revenue by loyalty tier
fig7 = px.bar(df.groupby('loyalty_tier')['gross_revenue'].sum().reset_index(),
              x='loyalty_tier', y='gross_revenue', title='Revenue by Loyalty Tier', color='loyalty_tier')
fig7.show()

# 8. Revenue by acquisition channel
fig8 = px.bar(df.groupby('acquisition_channel')['gross_revenue'].sum().reset_index(),
              x='acquisition_channel', y='gross_revenue', title='Revenue by Acquisition Channel', color='acquisition_channel')
fig8.show()

# 9. Revenue by product category
fig9 = px.bar(df.groupby('category')['gross_revenue'].sum().reset_index(),
              x='category', y='gross_revenue', title='Revenue by Category', color='category')
fig9.show()

# 10. Revenue by brand
fig10 = px.bar(df.groupby('brand')['gross_revenue'].sum().reset_index(),
               x='brand', y='gross_revenue', title='Revenue by Brand', color='brand')
fig10.show()

# ----------------------------
# 2️⃣ Quantity / Discount Analysis
# ----------------------------
# 11. Quantity by category
fig11 = px.bar(df.groupby('category')['quantity'].sum().reset_index(),
               x='category', y='quantity', title='Total Quantity Sold by Category', color='category')
fig11.show()

# 12. Quantity by brand
fig12 = px.bar(df.groupby('brand')['quantity'].sum().reset_index(),
               x='brand', y='quantity', title='Quantity Sold by Brand', color='brand')
fig12.show()

# 13. Discount applied distribution
fig13 = px.histogram(df, x='discount_applied', nbins=20, title='Discount Applied Distribution')
fig13.show()

# 14. Discount vs Revenue
fig14 = px.scatter(df, x='discount_applied', y='gross_revenue', title='Discount vs Revenue', trendline='ols')
fig14.show()

# 15. Quantity vs Revenue
fig15 = px.scatter(df, x='quantity', y='gross_revenue', title='Quantity vs Revenue', trendline='ols')
fig15.show()

# ----------------------------
# 3️⃣ Campaign & Refund Analysis
# ----------------------------
# 16. Revenue by campaign
fig16 = px.bar(df.groupby('campaign_id')['gross_revenue'].sum().reset_index(),
               x='campaign_id', y='gross_revenue', title='Revenue by Campaign')
fig16.show()

# 17. Number of refunds
fig17 = px.bar(df.groupby('refund_flag')['transaction_id'].count().reset_index(),
               x='refund_flag', y='transaction_id', title='Number of Refunds')
fig17.show()

# 18. Refunds by country
fig18 = px.bar(df[df['refund_flag']==1].groupby('country')['transaction_id'].count().reset_index(),
               x='country', y='transaction_id', title='Refunds by Country')
fig18.show()

# 19. Refunds by category
fig19 = px.bar(df[df['refund_flag']==1].groupby('category')['transaction_id'].count().reset_index(),
               x='category', y='transaction_id', title='Refunds by Category')
fig19.show()

# 20. Refunds by brand
fig20 = px.bar(df[df['refund_flag']==1].groupby('brand')['transaction_id'].count().reset_index(),
               x='brand', y='transaction_id', title='Refunds by Brand')
fig20.show()

# ----------------------------
# 4️⃣ Customer Signup Analysis
# ----------------------------
# 21. Customers by country
fig21 = px.bar(df.groupby('country')['customer_id'].nunique().reset_index(),
               x='country', y='customer_id', title='Number of Customers by Country')
fig21.show()

# 22. Customers by loyalty tier
fig22 = px.bar(df.groupby('loyalty_tier')['customer_id'].nunique().reset_index(),
               x='loyalty_tier', y='customer_id', title='Number of Customers by Loyalty Tier')
fig22.show()

# 23. Customers by acquisition channel
fig23 = px.bar(df.groupby('acquisition_channel')['customer_id'].nunique().reset_index(),
               x='acquisition_channel', y='customer_id', title='Customers by Acquisition Channel')
fig23.show()

# 24. Customers by age
fig24 = px.histogram(df, x='age', nbins=20, title='Customer Age Distribution')
fig24.show()

# 25. Customers by gender
fig25 = px.bar(df.groupby('gender')['customer_id'].nunique().reset_index(),
               x='gender', y='customer_id', title='Customers by Gender', color='gender')
fig25.show()

# ----------------------------
# 5️⃣ Product Launch & Price Analysis
# ----------------------------
# 26. Product base price distribution
fig26 = px.histogram(df, x='base_price', nbins=50, title='Product Base Price Distribution')
fig26.show()

# 27. Revenue by is_premium
fig27 = px.bar(df.groupby('is_premium')['gross_revenue'].sum().reset_index(),
               x='is_premium', y='gross_revenue', title='Revenue by Premium Product')
fig27.show()

# 28. Revenue by launch year
df['launch_year'] = pd.to_datetime(df['launch_date'], errors='coerce').dt.year
fig28 = px.bar(df.groupby('launch_year')['gross_revenue'].sum().reset_index(),
               x='launch_year', y='gross_revenue', title='Revenue by Product Launch Year')
fig28.show()

# 29. Quantity sold by launch year
fig29 = px.bar(df.groupby('launch_year')['quantity'].sum().reset_index(),
               x='launch_year', y='quantity', title='Quantity Sold by Launch Year')
fig29.show()

# 30. Revenue by category & country
fig30 = px.bar(df.groupby(['country','category'])['gross_revenue'].sum().reset_index(),
               x='category', y='gross_revenue', color='country', title='Revenue by Category & Country')
fig30.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Ensure timestamp is datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year_month'] = df['timestamp'].dt.to_period('M').astype(str)
df['weekday'] = df['timestamp'].dt.day_name()
df['launch_year'] = pd.to_datetime(df['launch_date'], errors='coerce').dt.year

# ----------------------------
# 31. Revenue by country & gender
# ----------------------------
fig31 = px.bar(df.groupby(['country','gender'])['gross_revenue'].sum().reset_index(),
               x='country', y='gross_revenue', color='gender', barmode='group',
               title='Revenue by Country & Gender')
fig31.show()

# ----------------------------
# 32. Revenue by category & gender
# ----------------------------
fig32 = px.bar(df.groupby(['category','gender'])['gross_revenue'].sum().reset_index(),
               x='category', y='gross_revenue', color='gender', barmode='group',
               title='Revenue by Category & Gender')
fig32.show()

# ----------------------------
# 33. Revenue by age groups
# ----------------------------
# Create age bins
bins = [18, 25, 35, 45, 55, 65, 100]
labels = ['18-24','25-34','35-44','45-54','55-64','65+']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)
age_group_rev = df.groupby('age_group')['gross_revenue'].sum().reset_index()

fig33 = px.bar(age_group_rev, x='age_group', y='gross_revenue', title='Revenue by Age Group')
fig33.show()

# ----------------------------
# 34. Revenue by weekday & category
# ----------------------------
fig34 = px.bar(df.groupby(['weekday','category'])['gross_revenue'].sum().reset_index(),
               x='weekday', y='gross_revenue', color='category', barmode='group',
               title='Revenue by Weekday & Category')
fig34.show()

# ----------------------------
# 35. Quantity sold by country & category
# ----------------------------
fig35 = px.bar(df.groupby(['country','category'])['quantity'].sum().reset_index(),
               x='category', y='quantity', color='country', barmode='group',
               title='Quantity Sold by Country & Category')
fig35.show()

# ----------------------------
# 36. Refunds by country & category
# ----------------------------
refund_df = df[df['refund_flag']==1]
fig36 = px.bar(refund_df.groupby(['country','category'])['transaction_id'].count().reset_index(),
               x='category', y='transaction_id', color='country', barmode='group',
               title='Refunds by Country & Category')
fig36.show()

# ----------------------------
# 37. Discount applied by category
# ----------------------------
fig37 = px.box(df, x='category', y='discount_applied', title='Discount Distribution by Category')
fig37.show()

# ----------------------------
# 38. Gross revenue vs quantity scatter
# ----------------------------
fig38 = px.scatter(df, x='quantity', y='gross_revenue', color='category',
                   size='discount_applied', title='Revenue vs Quantity (Size=Discount)')
fig38.show()

# ----------------------------
# 39. Revenue by launch year & category
# ----------------------------
fig39 = px.bar(df.groupby(['launch_year','category'])['gross_revenue'].sum().reset_index(),
               x='launch_year', y='gross_revenue', color='category', barmode='group',
               title='Revenue by Launch Year & Category')
fig39.show()

# ----------------------------
# 40. Premium vs Non-Premium Revenue by Category
# ----------------------------
fig40 = px.bar(df.groupby(['category','is_premium'])['gross_revenue'].sum().reset_index(),
               x='category', y='gross_revenue', color='is_premium', barmode='group',
               title='Revenue by Category & Premium Status')
fig40.show()

# ----------------------------
# 41. Top 10 brands by revenue
# ----------------------------
top_brands = df.groupby('brand')['gross_revenue'].sum().sort_values(ascending=False).head(10).reset_index()
fig41 = px.bar(top_brands, x='brand', y='gross_revenue', title='Top 10 Brands by Revenue')
fig41.show()

# ----------------------------
# 42. Top 10 categories by quantity sold
# ----------------------------
top_categories = df.groupby('category')['quantity'].sum().sort_values(ascending=False).head(10).reset_index()
fig42 = px.bar(top_categories, x='category', y='quantity', title='Top 10 Categories by Quantity Sold')
fig42.show()

# ----------------------------
# 43. Revenue trend for top 5 countries
# ----------------------------
top_countries = df.groupby('country')['gross_revenue'].sum().sort_values(ascending=False).head(5).index
top_country_trend = df[df['country'].isin(top_countries)].groupby(['year_month','country'])['gross_revenue'].sum().reset_index()

fig43 = px.line(top_country_trend, x='year_month', y='gross_revenue', color='country',
                title='Revenue Trend for Top 5 Countries', markers=True)
fig43.show()

# ----------------------------
# 44. Revenue trend for top 5 categories
# ----------------------------
top_cats = df.groupby('category')['gross_revenue'].sum().sort_values(ascending=False).head(5).index
top_cat_trend = df[df['category'].isin(top_cats)].groupby(['year_month','category'])['gross_revenue'].sum().reset_index()

fig44 = px.line(top_cat_trend, x='year_month', y='gross_revenue', color='category',
                title='Revenue Trend for Top 5 Categories', markers=True)
fig44.show()


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Ensure timestamp is datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year_month'] = df['timestamp'].dt.to_period('M').astype(str)
df['weekday'] = df['timestamp'].dt.day_name()
df['launch_year'] = pd.to_datetime(df['launch_date'], errors='coerce').dt.year

# ----------------------------
# 45. Heatmap: Revenue by Category & Month
# ----------------------------
cat_month_rev = df.groupby(['category','year_month'])['gross_revenue'].sum().reset_index()
fig45 = px.density_heatmap(cat_month_rev, x='year_month', y='category', z='gross_revenue',
                           color_continuous_scale='Viridis', title='Revenue Heatmap by Category & Month')
fig45.show()

# ----------------------------
# 46. Heatmap: Revenue by Country & Month
# ----------------------------
country_month_rev = df.groupby(['country','year_month'])['gross_revenue'].sum().reset_index()
fig46 = px.density_heatmap(country_month_rev, x='year_month', y='country', z='gross_revenue',
                           color_continuous_scale='Cividis', title='Revenue Heatmap by Country & Month')
fig46.show()

# ----------------------------
# 47. Treemap: Revenue by Country & Category
# ----------------------------
treemap_rev = df.groupby(['country','category'])['gross_revenue'].sum().reset_index()
fig47 = px.treemap(treemap_rev, path=['country','category'], values='gross_revenue',
                   title='Revenue Distribution by Country & Category', color='gross_revenue',
                   color_continuous_scale='RdBu')
fig47.show()

# ----------------------------
# 48. Sunburst: Revenue by Category & Brand
# ----------------------------
sunburst_rev = df.groupby(['category','brand'])['gross_revenue'].sum().reset_index()
fig48 = px.sunburst(sunburst_rev, path=['category','brand'], values='gross_revenue',
                    title='Revenue Distribution by Category & Brand')
fig48.show()

# ----------------------------
# 49. Scatter Matrix: Quantity, Revenue, Discount, Age
# ----------------------------
fig49 = px.scatter_matrix(df, dimensions=['quantity','gross_revenue','discount_applied','age'],
                          color='category', title='Scatter Matrix: Revenue, Quantity, Discount, Age')
fig49.show()

# ----------------------------
# 50. Bubble Chart: Revenue vs Quantity by Brand
# ----------------------------
brand_stats = df.groupby('brand').agg({'gross_revenue':'sum','quantity':'sum','discount_applied':'mean'}).reset_index()
fig50 = px.scatter(brand_stats, x='quantity', y='gross_revenue', size='discount_applied',
                   color='brand', hover_name='brand', title='Revenue vs Quantity by Brand (Bubble=Discount)')
fig50.show()

# ----------------------------
# 51. Stacked Area: Revenue by Category over Time
# ----------------------------
area_df = df.groupby(['year_month','category'])['gross_revenue'].sum().reset_index()
fig51 = px.area(area_df, x='year_month', y='gross_revenue', color='category',
                title='Revenue by Category Over Time')
fig51.show()

# ----------------------------
# 52. Line + Markers: Revenue Trend Top 5 Countries
# ----------------------------
top_countries = df.groupby('country')['gross_revenue'].sum().sort_values(ascending=False).head(5).index
top_country_df = df[df['country'].isin(top_countries)].groupby(['year_month','country'])['gross_revenue'].sum().reset_index()
fig52 = px.line(top_country_df, x='year_month', y='gross_revenue', color='country', markers=True,
                title='Revenue Trend for Top 5 Countries')
fig52.show()

# ----------------------------
# 53. Box Plot: Revenue Distribution by Loyalty Tier
# ----------------------------
fig53 = px.box(df, x='loyalty_tier', y='gross_revenue', color='loyalty_tier',
               title='Revenue Distribution by Loyalty Tier')
fig53.show()

# ----------------------------
# 54. Violin Plot: Quantity Distribution by Category
# ----------------------------
fig54 = px.violin(df, x='category', y='quantity', color='category', box=True, points='all',
                  title='Quantity Distribution by Category')
fig54.show()

# ----------------------------
# 55. Correlation Heatmap (Numeric Features)
# ----------------------------
import plotly.figure_factory as ff
numeric_cols = ['quantity','discount_applied','gross_revenue','age','base_price','is_premium']
corr_matrix = df[numeric_cols].corr().round(2)
fig55 = ff.create_annotated_heatmap(z=corr_matrix.values, x=list(corr_matrix.columns), y=list(corr_matrix.index),
                                    colorscale='Viridis', showscale=True)
fig55.update_layout(title='Correlation Heatmap (Numeric Features)')
fig55.show()

# ----------------------------
# 56. Time Series Decomposition (Optional, requires statsmodels)
# ----------------------------
from statsmodels.tsa.seasonal import seasonal_decompose
ts = df.groupby('timestamp')['gross_revenue'].sum().asfreq('D').fillna(0)
result = seasonal_decompose(ts, model='additive', period=30)
fig56 = go.Figure()
fig56.add_trace(go.Scatter(y=result.trend, name='Trend'))
fig56.add_trace(go.Scatter(y=result.seasonal, name='Seasonality'))
fig56.add_trace(go.Scatter(y=result.resid, name='Residual'))
fig56.update_layout(title='Revenue Time Series Decomposition')
fig56.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
import matplotlib.pyplot as plt

# -----------------------------
# Load & Merge datasets
# -----------------------------
path = "/kaggle/input/marketing-and-e-commerce-analytics-dataset/"

products = pd.read_csv(path + "products.csv")
customers = pd.read_csv(path + "customers.csv")
transactions = pd.read_csv(path + "transactions.csv")

df = transactions.merge(customers, on="customer_id", how="left")
df = df.merge(products, on="product_id", how="left")

# -----------------------------
# Ensure datetime
# -----------------------------
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['signup_date'] = pd.to_datetime(df['signup_date'])
df['launch_date'] = pd.to_datetime(df['launch_date'], errors='coerce')

# -----------------------------
# Handle Missing Values
# -----------------------------
numeric_cols = ['quantity','discount_applied','age','base_price','gross_revenue']
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].median())

categorical_cols = ['country','gender','loyalty_tier','acquisition_channel','category','brand','is_premium']
for col in categorical_cols:
    df[col] = df[col].fillna("Unknown")

# -----------------------------
# Feature Engineering
# -----------------------------
# Temporal Features
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['day'] = df['timestamp'].dt.day
df['weekday'] = df['timestamp'].dt.weekday
df['hour'] = df['timestamp'].dt.hour

# Product & customer features
df['product_age_days'] = (df['timestamp'] - df['launch_date']).dt.days
df['product_age_days'] = df['product_age_days'].fillna(df['product_age_days'].median())

df['days_since_signup'] = (df['timestamp'] - df['signup_date']).dt.days
df['days_since_signup'] = df['days_since_signup'].fillna(df['days_since_signup'].median())

# Lag Features: sort by customer
df = df.sort_values(by=['customer_id','timestamp'])

df['customer_prev_revenue'] = df.groupby('customer_id')['gross_revenue'].shift(1).fillna(0)
df['customer_cum_revenue'] = df.groupby('customer_id')['gross_revenue'].cumsum() - df['gross_revenue']
df['customer_prev_quantity'] = df.groupby('customer_id')['quantity'].shift(1).fillna(0)

# Encode Categorical Features
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

# -----------------------------
# Features and Target
# -----------------------------
target = 'gross_revenue'
features = [
    'quantity','discount_applied','age','days_since_signup','product_age_days',
    'customer_prev_revenue','customer_cum_revenue','customer_prev_quantity',
    'year','month','day','weekday','hour'
] + categorical_cols

X = df[features]
y = df[target]

# -----------------------------
# Train-Validation Split
# -----------------------------
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------
# LightGBM Dataset
# -----------------------------
lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_val, y_val, reference=lgb_train)

# -----------------------------
# LightGBM Parameters
# -----------------------------
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'learning_rate': 0.05,
    'num_leaves': 131,
    'max_depth': -1,
    'min_data_in_leaf': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

# -----------------------------
# Train LightGBM Model
# -----------------------------
model = lgb.train(
    params,
    lgb_train,
    num_boost_round=1000,
    valid_sets=[lgb_train, lgb_val],
)

# -----------------------------
# Predictions
# -----------------------------
y_train_pred = model.predict(X_train, num_iteration=model.best_iteration)
y_val_pred = model.predict(X_val, num_iteration=model.best_iteration)

# -----------------------------
# Metrics
# -----------------------------
def print_metrics(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"\n{dataset_name} Metrics:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R^2: {r2:.2f}")

print_metrics(y_train, y_train_pred, "Train")
print_metrics(y_val, y_val_pred, "Validation")

# -----------------------------
# Feature Importance
# -----------------------------
lgb.plot_importance(model, max_num_features=20, importance_type='gain', figsize=(10,6))
plt.title("Top 20 Feature Importance")
plt.show()


In [ ]:
from sklearn.linear_model import LinearRegression
from tensorflow.keras.layers import Dense, Dropout


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# -----------------------------
# Load & Merge datasets
# -----------------------------

# -----------------------------
# Train Linear Regression
# -----------------------------
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

# -----------------------------
# Predictions
# -----------------------------
y_train_pred = lr_model.predict(X_train)
y_val_pred = lr_model.predict(X_val)

# -----------------------------
# Metrics
# -----------------------------
def print_metrics(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"\n{dataset_name} Metrics:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R^2: {r2:.2f}")

print_metrics(y_train, y_train_pred, "Train")
print_metrics(y_val, y_val_pred, "Validation")

# -----------------------------
# Feature Importance (Coefficients)
# -----------------------------
coefs = pd.Series(lr_model.coef_, index=X_train.columns)
coefs = coefs.sort_values(key=abs, ascending=False)[:20]

plt.figure(figsize=(10,6))
coefs.plot(kind='barh')
plt.title("Top 20 Feature Coefficients (Linear Regression)")
plt.gca().invert_yaxis()
plt.show()


In [ ]:
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential


# -----------------------------
# Load & Merge datasets
# -----------------------------
# -----------------------------
# Build ANN Model
# -----------------------------
model = Sequential([
    Dense(128, input_dim=X_train_scaled.shape[1], activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(1, activation='linear')  # Regression output
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# -----------------------------
# Train ANN
# -----------------------------
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=50,
    batch_size=512,
    verbose=1
)

# -----------------------------
# Predictions
# -----------------------------
y_train_pred = model.predict(X_train_scaled).flatten()
y_val_pred = model.predict(X_val_scaled).flatten()

# -----------------------------
# Metrics
# -----------------------------
def print_metrics(y_true, y_pred, dataset_name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"\n{dataset_name} Metrics:")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R^2: {r2:.2f}")

print_metrics(y_train, y_train_pred, "Train")
print_metrics(y_val, y_val_pred, "Validation")

# -----------------------------
# Plot Training Loss
# -----------------------------
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('MSE Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()
